In [1]:
filename = '0_arch.js'

In [2]:
import subprocess
import textwrap

js_wrapper = textwrap.dedent("""
// file system
const fs = require('fs'); 
const code = fs.readFileSync('"""+filename+"""', 'utf8');

// mock object
const bot = { interrupt_code: false };
const world = {
  getPosition: () => ({ x: 0, y: 0, z: 0 }), // StartXYZ
  getBlockAtPosition: () => ({ name: 'air' })
};
function log(){}

// placeBlock override
const skills = {
  breakBlockAt: async () => {},
  placeBlock: async (_bot, block, x, y, z) => {
    console.log(JSON.stringify({x,y,z}));
  }
};

(async () => {
  const f = eval(code);
  if (typeof f === 'function') {
    await f(bot);
  }
})();
""")

result = subprocess.run(
    ["node", "-e", js_wrapper],
    capture_output=True,
    text=True
)

# parsing
coords = []
for line in result.stdout.splitlines():
    try:
        coords.append(eval(line))
    except:
        pass

coords


[{'x': -7, 'y': 0, 'z': -1},
 {'x': -7, 'y': 1, 'z': -1},
 {'x': -7, 'y': 2, 'z': -1},
 {'x': -7, 'y': 3, 'z': -1},
 {'x': -7, 'y': 0, 'z': 0},
 {'x': -7, 'y': 1, 'z': 0},
 {'x': -7, 'y': 2, 'z': 0},
 {'x': -7, 'y': 3, 'z': 0},
 {'x': -7, 'y': 0, 'z': 1},
 {'x': -7, 'y': 1, 'z': 1},
 {'x': -7, 'y': 2, 'z': 1},
 {'x': -7, 'y': 3, 'z': 1},
 {'x': 6, 'y': 0, 'z': -1},
 {'x': 6, 'y': 1, 'z': -1},
 {'x': 6, 'y': 2, 'z': -1},
 {'x': 6, 'y': 3, 'z': -1},
 {'x': 6, 'y': 0, 'z': 0},
 {'x': 6, 'y': 1, 'z': 0},
 {'x': 6, 'y': 2, 'z': 0},
 {'x': 6, 'y': 3, 'z': 0},
 {'x': 6, 'y': 0, 'z': 1},
 {'x': 6, 'y': 1, 'z': 1},
 {'x': 6, 'y': 2, 'z': 1},
 {'x': 6, 'y': 3, 'z': 1},
 {'x': -6, 'y': 2, 'z': 0},
 {'x': -5, 'y': 2, 'z': 0},
 {'x': -4, 'y': 2, 'z': 0},
 {'x': -3, 'y': 2, 'z': 0},
 {'x': -3, 'y': 1, 'z': 0},
 {'x': -2, 'y': 2, 'z': 0},
 {'x': -2, 'y': 1, 'z': 0},
 {'x': -1, 'y': 2, 'z': 0},
 {'x': -1, 'y': 1, 'z': 0},
 {'x': -1, 'y': 0, 'z': 0},
 {'x': 0, 'y': 2, 'z': 0},
 {'x': 0, 'y': 1, 'z': 0}

In [3]:
def _to_int(v): return int(str(v).strip())

def _xyz_list(coords):
    xs, ys, zs = [], [], []
    for c in coords:
        x, y, z = _to_int(c["x"]), _to_int(c["y"]), _to_int(c["z"])
        xs.append(x); ys.append(y); zs.append(z)
    return xs, ys, zs

def eval(coords):
    xs, ys, zs = _xyz_list(coords)
    if not coords:
        return False, "Count mismatch: 0 (expected >0)"

    mean_y = sum(ys) / len(ys)
    minx, maxx = min(xs), max(xs)
    minz, maxz = min(zs), max(zs)
    cx = round((minx + maxx) / 2)
    cz = round((minz + maxz) / 2)

    # 1) 중심선(x=cx, z=cz)에서의 최대 높이가 전체 평균 이상인가?
    max_y_on_center_x = max(y for x,y in zip(xs,ys) if x == cx)
    max_y_on_center_z = max(y for z,y in zip(zs,ys) if z == cz)
    cond1 = (max_y_on_center_x >= mean_y) and (max_y_on_center_z >= mean_y)

    # 2) 정확한 중심 기둥(x=cx AND z=cz)의 모든 y가 평균 이상인가?
    center_col = [y for x,y,z in zip(xs,ys,zs) if x == cx and z == cz]
    cond2 = len(center_col) > 0 and all(y >= mean_y for y in center_col)

    # 3) 가장자리(x=min/max, z=min/max)의 y가 평균 미만인가?
    edge_y = [y for x,y in zip(xs,ys) if x in (minx,maxx)]
    edge_y += [y for z,y in zip(zs,ys) if z in (minz,maxz)]
    cond3 = len(edge_y) > 0 and all(y < mean_y for y in edge_y)

    ok = cond1 and cond2 and cond3
    return ok, f"arch={ok} | {cond1} {cond2} {cond3}"


# 사용 예:
ok, msg = eval(coords)
print(ok, msg)


False arch=False | True False False
